In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import string
import os
import pandas as pd
import datetime
import numpy as np
import statsmodels.api as sm
import math
from scipy.linalg import svd
#import Functions as Fun
import scipy.linalg as la
import scipy.sparse.linalg as sla

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
###Some parameters
mu=0.2;
tol=1e-10;

In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
####for training
nums= 30
start_dates = [pd.to_datetime('1957-03-01')+pd.offsets.DateOffset(years=x) for x in range(nums) ]

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
####f_gradh
def f_gradh(w,X,y,mu):
    r = np.matmul(X,w)-y['RET']
    ind0 = np.where(abs(r)<= mu)
    ind0 = pd.Series(np.asarray(ind0)[0])
    ind1 = np.where(r>mu)
    ind1 = pd.Series(np.asarray(ind1)[0])

    indf1 = np.where(r< (-mu))
    indf1 = pd.Series(np.array(indf1)[0])
    grad = np.matmul(np.transpose(X.iloc[ind0,:]),(np.matmul(X.iloc[ind0,:],w)-y.iloc[ind0,0]))+np.matmul(np.transpose(mu*X.iloc[ind1,:]),np.ones(len(ind1)))-np.matmul(np.transpose(mu*X.iloc[indf1,:]),np.ones(len(indf1)))
    return grad

In [ ]:
### proximal_l   only for lasso

def proximalH_l(groups,nc,xtest,mtrain,ytest,w,X,y,mu,tol,L,l2,func):
    w= np.asarray(w[0])
    dim = X.shape[0]
    max_iter = 3000
    gamma= 1/L
    l1 = l2

    v= w
    yhatbig1=np.matmul(xtest,w) + mtrain.values
    r20=lossh(yhatbig1,ytest,mu)
    for t in range(max_iter):
        vold=v
        w_perv=w
        w= np.asarray(v-(gamma*f_gradh(v,X,y,mu)))
        mu1=l1*gamma
        w=func(groups,nc,w,mu1)
        w = np.asarray([float(w1) for w1 in w])
        v=w + t/(t+3)*(w-w_perv)
        if ((np.linalg.norm(v-vold)**2) < ((np.linalg.norm(vold)**2)*tol) or np.sum(abs(v-vold))==0):
            break
    return v

In [ ]:
def soft_threshode(groups,nc,w,mu):
    val =[float(np.sign(w1)*np.max(np.abs(w1)-0.5*mu,0))/(1+0.5*mu) for w1 in w]
    return np.array(val).reshape((len(val),1))

In [ ]:
####lossh
def lossh(y,yhat,mu):
    r= abs(yhat['RET']-y)
    l= np.zeros(len(r))
    ind = (r>mu)
    l[ind] = 2*mu*r[ind]-mu*mu
    ind = (r<= mu)
    l[ind]  = r[ind]*r[ind]
    return np.mean(l)

In [ ]:
for m in range(len(start_dates)):

    start_date = start_dates[m]

    end_date =  start_date + pd.offsets.DateOffset(years=18)

    ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytrain = Y_all[ind_training]

    xtrain = X_all[ind_training]

    ####for validation

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=12)

    ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytest = Y_all[ind_vali]
    xtest= X_all[ind_vali]

    #####for test  oos

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=1)
    ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
    #print(type(ind_test))
    yoos = Y_all[ind_test]
    xoos= X_all[ind_test]

    #######Monthly Demean %%%
    ytrain_demean= ytrain - np.mean(ytrain);
    ytest_demean=ytest-np.mean(ytest);
    mtrain=np.mean(ytrain);
    mtest=np.mean(ytest);

    sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
    for i in range(len(xtrain.columns)):
        s=np.std(xtrain.iloc[:,[i]])
        colnames = xtrain.columns
        if s.values > 0:
            colname = colnames[i]
            xtrain.loc[:,colname]=xtrain[colname]/s.values
            xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
            xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
            sd[i]=s.values
    XX =np.matmul(np.transpose(xtrain),xtrain)
    U,S,V=svd(XX)
    #### singular value decomposition such that XX=U*S*V'
    L=S[0]
    ###disp 'Lasso L = '
    ####disp(L)
    Y=ytrain_demean;
    XY= np.matmul(np.transpose(xtrain),Y)
    groups=0
    nc=0
            
    #### Start to Train %%%
   
    lamv=np.arange(-2,4,0.1)
    alpha=1
    r=np.zeros((3,len(lamv)))
    y_ret=ytest['RET'].reset_index(drop=True)
    y_ret1=ytrain['RET'].reset_index(drop=True)
    y_ret2=yoos['RET'].reset_index(drop=True)


    for j in range(len(lamv)):
        l2=10**lamv[j]
        ela = ElasticNet(alpha=l2)
        yhatres= ela.fit(xtrain,ytrain_demean)
        yhatbig1= yhatres.predict(xtest)+ mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        #prediction for oos
        r[0,j]=1-sum((yhatbig1-y_ret)**2)/sum((y_ret-mtrain.values)**2) # oos r2
        yhatbig1=yhatres.predict(xtrain) + mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        r[1,j]=1-(yhatbig1-y_ret1).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
        yhatbig1=yhatres.predict(xoos) + mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        r[2,j]=1-(yhatbig1-y_ret2).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
        


    r2_oos=r[2,np.argmax(r[0,:])]
    r2_is=r[1,np.argmax(r[0,:])]
    l2=10**lamv[np.argmax(r[0,:])]
    ela = ElasticNet(alpha=l2)
    yhatres= ela.fit(xtrain,ytrain_demean)
    b=pd.DataFrame(yhatres.coef_)

    print('ElasticNet R2 : '+np.str0(r2_oos))
    print('ElasticNet R2_is : '+np.str0(r2_is))
    
    b=proximalH_l(groups,nc,xtest,mtrain,ytest,b,xtrain,ytrain_demean,mu,tol,L,l2,soft_threshode)
    yhatbig1 = np.matmul(xoos,b)+mtrain.values
    r2_oos=1- np.linalg.norm(yhatbig1-yoos['RET'])**2 / np.linalg.norm((yoos['RET']-mtrain.values))**2
    yhatbig1 = np.matmul(xtrain,b) + mtrain.values
    r2_is = 1 - np.linalg.norm(yhatbig1-ytrain['RET'])**2 / np.linalg.norm((ytrain['RET']-mtrain.values))**2
    b=pd.DataFrame(b)
    print('ElasticNet+H R2 : '+np.str0(r2_oos) )

<!--
ElasticNet R2 : 0.5618758011505414
ElasticNet R2_is : -2.220446049250313e-15
ElasticNet+H R2 : -13121.323802022502
ElasticNet R2 : 0.791334380349793
ElasticNet R2_is : -8.215650382226158e-15
ElasticNet+H R2 : -113.37234414760036
ElasticNet R2 : 0.8135787276602633
ElasticNet R2_is : -1.1546319456101628e-14
ElasticNet+H R2 : -91.19080867215344
ElasticNet R2 : 0.6712708722341254
ElasticNet R2_is : -3.774758283725532e-15
ElasticNet+H R2 : -15.793990883867338
ElasticNet R2 : 0.6175672682393455
ElasticNet R2_is : -1.532107773982716e-14
ElasticNet+H R2 : -21.52230169537312
ElasticNet R2 : 0.8836687807482796
ElasticNet R2_is : -4.440892098500626e-15
ElasticNet+H R2 : -18.25769669932044
ElasticNet R2 : 0.8815876009027938
ElasticNet R2_is : 0.00020507051532769882
ElasticNet+H R2 : -26.26540468678037
ElasticNet R2 : 0.8765587519765801
ElasticNet R2_is : 0.001220492915689042
ElasticNet+H R2 : -5.495280831041644
ElasticNet R2 : 0.8988952383748445
ElasticNet R2_is : 4.9960036108132044e-15
ElasticNet+H R2 : -364.8598241324494
ElasticNet R2 : 0.8999340719197466
ElasticNet R2_is : 0.0002704780123873318
ElasticNet+H R2 : -1027.9753459091569
ElasticNet R2 : 0.865727115062263
ElasticNet R2_is : -1.9761969838327786e-14
ElasticNet+H R2 : -854.1775414685286
ElasticNet R2 : 0.7540878589259792
ElasticNet R2_is : 0.0004816581349025295
ElasticNet+H R2 : -340.529106996444
ElasticNet R2 : 0.7126218628264784
ElasticNet R2_is : 0.0003371469231333579
ElasticNet+H R2 : -33.85961171256544
ElasticNet R2 : 0.7481664079562618
ElasticNet R2_is : 0.0004619973591408755
ElasticNet+H R2 : -1.6036605062709004
ElasticNet R2 : 0.8404811598033325
ElasticNet R2_is : 1.4886460452667727e-05
ElasticNet+H R2 : -0.7569714690251894
ElasticNet R2 : 0.8826477189068416
ElasticNet R2_is : 0.00016391367003343316
ElasticNet+H R2 : -0.21870007621187404
ElasticNet R2 : 0.9268850337157866
ElasticNet R2_is : 1.942890293094024e-14
ElasticNet+H R2 : -0.304601033459343
ElasticNet R2 : 0.9707250199351023
ElasticNet R2_is : 0.00035364970119189376
ElasticNet+H R2 : -0.32248926044618154
ElasticNet R2 : 0.9668228418145308
ElasticNet R2_is : 0.0003691852151842534
ElasticNet+H R2 : -0.12132326718114128
ElasticNet R2 : 0.9669078058373662
ElasticNet R2_is : -2.4424906541753444e-14
ElasticNet+H R2 : -0.0594399269676853
ElasticNet R2 : 0.9677117757903464
ElasticNet R2_is : 4.1300296516055823e-14
ElasticNet+H R2 : -0.0007711499818248146
ElasticNet R2 : 0.935613606584905
ElasticNet R2_is : 3.3721166043720885e-05
ElasticNet+H R2 : 0.06300133574686329
ElasticNet R2 : 0.9255911956162547
ElasticNet R2_is : 2.7533531010703882e-14
ElasticNet+H R2 : -0.07202489821094837
ElasticNet R2 : 0.9773278842816471
ElasticNet R2_is : -5.5067062021407764e-14
ElasticNet+H R2 : -0.02259912775478634
ElasticNet R2 : 0.9681833508388836
ElasticNet R2_is : 4.809087418067293e-05
ElasticNet+H R2 : 0.0051879715355065414
ElasticNet R2 : 0.9807186885727028
ElasticNet R2_is : 1.4099832412739488e-14
ElasticNet+H R2 : -0.04816241710728475
ElasticNet R2 : 0.9823593490279904
ElasticNet R2_is : 8.588274160148401e-05
ElasticNet+H R2 : 0.0003337987212145288
ElasticNet R2 : 0.9861978426740109
ElasticNet R2_is : -2.220446049250313e-14
ElasticNet+H R2 : -0.01987068025980987
ElasticNet R2 : 0.982101122170377
ElasticNet R2_is : -3.774758283725532e-15
ElasticNet+H R2 : 0.020377712284700977
ElasticNet R2 : 0.9813495401178732
ElasticNet R2_is : 0.00048718748368536957
ElasticNet+H R2 : -0.02952531772544109